# Importing libraries

In [ ]:
import warnings
import requests
from io import StringIO
# Libraries for visualization
import matplotlib.pyplot as plt
import seaborn as sns
# Libraries to handle data
import numpy as np
import pandas as pd
# Libraries for natural language processing re (regular expressions) and nltk (NLP)
import re
import nltk
# Tools for NLP like stopwords, stemmer, vectorizer, encoder
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
# Model Training Tools
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
# Modelos de Random Forest y KNN (Clasificadores)
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
# Tools for evaluation
from matplotlib.colors import ListedColormap
from sklearn import metrics
from sklearn.metrics import plot_confusion_matrix


## Data load

The team considers it necessary to mention that a generic data set "spam.csv" is used as a model test, which includes paragraphs or sentences with a variety of words (size).

In [ ]:
data = pd.read_csv("C:/spam.csv", encoding = "ISO-8859-1")
data.info()

Dropping irrelevant columns

In [ ]:
to_drop = ["Unnamed: 2","Unnamed: 3","Unnamed: 4"]
data = data.drop(data[to_drop], axis=1)
# Rename Columns
data.rename(columns = {"v1":"Target", "v2":"Text"}, inplace = True)
data.head()

# Feature engineering

The code in this section represents the exploration of features and creation of new assets in the data like:
* No_of_Characters: Number of characters
* No_of_Words: Number of words
* No_of_sentence: Number of sentences

In [ ]:
#Dowload of Punk Sentence Tokenizer
# Tokenizer divides text in sentences
nltk.download('punkt')

In [ ]:
#Adding new columns (features)
data["No_of_Characters"] = data["Text"].apply(len)
data["No_of_Words"] = data.apply(
    lambda row: nltk.word_tokenize(row["Text"]), axis=1).apply(len) #Tokenization of Text words
data["No_of_sentence"] = data.apply(
    lambda row: nltk.sent_tokenize(row["Text"]), axis=1).apply(len) #Tokenization of  Text sentences

data.describe().T


Even as just a test , the data could contain non-typical values 

In [ ]:
plt.figure(figsize=(12, 8))
cols= ["#E1F16B", "#E598D8"]
fg = sns.pairplot(data=data, hue="Target", palette=cols)
plt.show(fg)


We set the number of characters (due to so much non-typical values) to less than 350 per row

In [ ]:
#Eliminando los valores atípicos
data = data[(data["No_of_Characters"] < 350)]
data.shape

## Data preprocessing

The data cleansing process in Natural Language Processing (NLP) is crucial. Without it, the computer does not understand the text and interprets it as a meaningless set of symbols. For more complex data processing we require data cleansing compulsorily.

* First stage: Extract alphabetical characters then punctuations and numbers
* Second stage: Lowercase characters

In [ ]:
# Clean function definition
def Clean(Text):
    sms = re.sub('[^a-zA-Z]', ' ', Text) #The regular expression of [^a-zA-Z] replaces non alphabetical into blanks
    sms = sms.lower() #Lowercase convertion
    sms = sms.split()
    sms = ' '.join(sms)
    return sms

#New column for clean text
data["Clean_Text"] = data["Text"].apply(Clean)

# Tokenization

Tokenization is about spliting complex data in little units called tokens . In this case , pharagraphs into words and sentences into words

In [ ]:
# Applying lambda function for tokenization
data["Tokenize_Text"]=data.apply(lambda row: nltk.word_tokenize(row["Clean_Text"]), axis=1)

## Stopwords

Stopwords are words with high frecuency but with no special contribution to NLP (like , no , few) . NLTK library has default stopwords for elimination

In [ ]:
# Stopwords delete function
def remove_stopwords(text):
    stop_words = set(stopwords.words("english"))
    filtered_text = [word for word in text if word not in stop_words]
    return filtered_text

data["Nostopword_Text"] = data["Tokenize_Text"].apply(remove_stopwords)

## Lemmatization

This process is about obtaining the core of the words , but ensuring the correct translation

In [ ]:
nltk.download('wordnet')
nltk.download('omw-1.4')


In [ ]:
# Defining the lemmatizer parameter
lemmatizer = WordNetLemmatizer()

# Function for lemmatize a word
def lemmatize_word(text):
    lemmas = [lemmatizer.lemmatize(word, pos ='v') for word in text]
    return lemmas

#New column with lemmatize text
data["Lemmatized_Text"] = data["Nostopword_Text"].apply(lemmatize_word)


## Vectorization

TF-IDF in NLP stands for Term Frequency: Inverse Document Frequency. In NLP, clean data must be converted to a numeric format where each word is represented by a matrix. This is also known as word embedding or word vectorization.

Term Frequency (TF) = (Frequency of a term in the document)/(Total number of terms in the documents) Inverse Document Frequency (IDF) = log ((Total number of documents)/(Number of documents with the term t)) I'll use TfidfVectorizer() to vectorize the preprocessed data.

Steps in Vectorization:

* Creation of a lemmatized text corpus.
* Convert the corpus to vector form
* Label encoding of classes in Target

In [ ]:
# Function for create corpus (More vectorized text)
corpus = []
for i in data["Lemmatized_Text"]:
    msg = ' '.join([row for row in i])
    corpus.append(msg)


Converting text into numbers by Vectorizer

In [ ]:
tfidf = TfidfVectorizer()
#Creating an X data section (Independent variables)
X = tfidf.fit_transform(corpus).toarray()

Creating the Y variable , the target of spam or non-spam

In [ ]:
label_encoder = LabelEncoder()
data["Target"] = label_encoder.fit_transform(data["Target"])

## Contruction of classifiers

Defining the Y , the target and sets for training

In [ ]:
y = data["Target"] 
# Train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### Random Forest Model

In [ ]:
rf = RandomForestClassifier(n_estimators=300,max_depth=10,min_samples_split=5,min_samples_leaf=2,max_features='auto',random_state=42)
rf.fit(X_train,y_train)

### KNN Model

In [ ]:
knn = KNeighborsClassifier(n_neighbors=5,weights='uniform',n_jobs=-1,algorithm='auto')
knn.fit(X_train,y_train)

# Model Testing

In [ ]:
#Creating metrics for both models
precision =[]
recall =[]
f1_score = []

#List of models
classifiers = [rf,knn]

# Evaluating each model
for i in classifiers:
    pred_train = i.predict(X_train)
    pred_test = i.predict(X_test)
    prec = metrics.precision_score(y_test, pred_test)
    recal = metrics.recall_score(y_test, pred_test)
    f1_s = metrics.f1_score(y_test, pred_test)

  
    # Results
    precision.append(prec)
    recall.append(recal)
    f1_score.append(f1_s)

Now for the testing we shall initiate a list of the metrics

In [ ]:
evaluation = {'Precision':precision,
'Recall':recall,
'F1score':f1_score}
# Creando dataframe en pandas
Results = pd.DataFrame(evaluation, index =[ "RandomForest", "KNeighbours"])

The results dataframe is shown in colors

In [ ]:
cmap2 = ListedColormap(["#E2CCFF", "#E598D8"])
Results.style.background_gradient(cmap=cmap2)


Finally , a graphic of the evaluation for both models

In [ ]:
cmap = ListedColormap(["#E1F16B", "#E598D8"])
fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(15, 10))

for cls, ax in zip(classifiers, axes.flatten()):
    plot_confusion_matrix(cls,
                          X_test,
                          y_test,
                          ax=ax,
                          cmap=cmap,
                          )
    ax.title.set_text(type(cls).__name__)
plt.tight_layout()
plt.show()


Note: Conclusions of testing are documented in the respective data issue 